<a href="https://colab.research.google.com/github/SimoneMonastero/Pink_Horizons/blob/main/Classification/1_classifier_IMU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CLASSIFICATION TASK

In [1]:
import numpy as np #used a lot in signal processing
import pandas as pd #used with spreadshits
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression #read thedocumantation about it
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import recall_score

# Setting the seed of the random generator
SEED = 42
n_folds = 5;


In [2]:
X = pd.read_excel('dataset.xlsx')
X=X.dropna(axis=0)
X.head(5)

,Subjects,walking_percentage,total_steps,stride_time,valid_bouts,avg_range,total_minutes,FSST,labels
0,CO-003,0.245186,1298,1.024000,7,1.365044,11.033333,12.50,0
1,CO-004,2.071489,9673,1.137778,42,1.191708,93.216667,11.53,0
2,CO-005,5.103469,18690,1.137778,65,1.484476,221.750000,13.70,0
3,CO-006,1.560376,5561,1.137778,28,1.323149,70.216667,10.12,0
4,CO-007,1.259634,5676,1.024000,20,1.709768,56.683333,8.34,0


In [3]:
features_FA = list(X.columns) #with df.columns extract the name of all the
#columns and with list we create a list with all the names


In [4]:
# List all the columns you want to drop
columns_to_drop = ['total_minutes', 'valid_bouts', 'FSST', 'Subjects', 'labels']

# Create the features DataFrame by dropping all of them in one go
# The original DataFrame X remains untouched, which is a safer practice.
X_features = X.drop(columns=columns_to_drop, axis=1)

display(X_features.head())

,walking_percentage,total_steps,stride_time,avg_range
0,0.245186,1298,1.024000,1.365044
1,2.071489,9673,1.137778,1.191708
2,5.103469,18690,1.137778,1.484476
3,1.560376,5561,1.137778,1.323149
4,1.259634,5676,1.024000,1.709768


In [5]:
Y = X['labels']
Y.head()

,labels
0,0
1,0
2,0
3,0
4,0


Questo codice concatena i due dataframe `FA` e `CO` in un nuovo dataframe chiamato `X`. L'argomento `ignore_index=True` assicura che il nuovo dataframe `X` abbia un indice continuo e non mantenga gli indici originali di `FA` e `CO` che potrebbero essere duplicati.

In [6]:
clf = LogisticRegression(max_iter=1000, class_weight='balanced')


cv = KFold(n_splits=n_folds, shuffle=True, random_state=SEED)

for train_index, test_index in cv.split(X_features):
    print("Train:", train_index, " Test:", test_index)

Train: [ 1  2  4  6  7  8  9 10 11 14 15 16 17 18 20 21 22 23 24 25 27 28 29 30
 31 32 34 35 37 38 39 40 41 42 43 45 46 47 48 49 51 52 53 55]  Test: [ 0  3  5 12 13 19 26 33 36 44 50 54]
Train: [ 0  1  2  3  5  7  9 10 11 12 13 14 15 16 18 19 20 21 22 23 24 25 26 28
 29 31 32 33 35 36 37 38 39 40 42 43 44 45 48 49 50 51 53 54 55]  Test: [ 4  6  8 17 27 30 34 41 46 47 52]
Train: [ 0  1  2  3  4  5  6  7  8 10 12 13 14 17 18 19 20 21 22 23 26 27 28 29
 30 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 50 51 52 54 55]  Test: [ 9 11 15 16 24 25 31 32 48 49 53]
Train: [ 0  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 22 24 25 26 27
 28 30 31 32 33 34 36 38 41 42 44 46 47 48 49 50 51 52 53 54 55]  Test: [ 1  2 21 23 29 35 37 39 40 43 45]
Train: [ 0  1  2  3  4  5  6  8  9 11 12 13 15 16 17 19 21 23 24 25 26 27 29 30
 31 32 33 34 35 36 37 39 40 41 43 44 45 46 47 48 49 50 52 53 54]  Test: [ 7 10 14 18 20 22 28 38 42 51 55]


In [7]:
score = cross_validate(clf, X=X_features, y=Y, cv=cv, return_train_score=True, return_estimator=True, scoring = 'roc_auc')
print("This is the score object:")
print (score)
print("Average AUC test set:", np.mean(score['test_score']))

This is the score object:
{'fit_time': array([0.05843425, 0.02198625, 0.02499199, 0.02244687, 0.01983905]), 'score_time': array([0.00806022, 0.00444603, 0.00410771, 0.00484729, 0.00410795]), 'estimator': [LogisticRegression(class_weight='balanced', max_iter=1000), LogisticRegression(class_weight='balanced', max_iter=1000), LogisticRegression(class_weight='balanced', max_iter=1000), LogisticRegression(class_weight='balanced', max_iter=1000), LogisticRegression(class_weight='balanced', max_iter=1000)], 'test_score': array([0.77142857, 0.6       , 0.70833333, 0.7       , 0.71428571]), 'train_score': array([0.71581197, 0.76748971, 0.73      , 0.71218487, 0.74696356])}
Average AUC test set: 0.6988095238095239


In [8]:
from sklearn.metrics import recall_score

sensitivities = []
specificities = []

for i, (train_index, test_index) in enumerate(cv.split(X_features, Y)):
    # Get the fitted estimator for the current fold
    fold_estimator = score['estimator'][i]

    # Get the test data for the current fold
    X_test_fold = X_features.iloc[test_index]
    y_test_fold = Y.iloc[test_index]

    # Make predictions on the test set for the current fold
    y_pred_fold = fold_estimator.predict(X_test_fold)

    # Calculate sensitivity (recall for class 1)
    sensitivity = recall_score(y_test_fold, y_pred_fold, pos_label=1)
    sensitivities.append(sensitivity)

    # Calculate specificity (recall for class 0)
    specificity = recall_score(y_test_fold, y_pred_fold, pos_label=0)
    specificities.append(specificity)

print("Average Sensitivity after cross-validation:", np.mean(sensitivities))
print("Average Specificity after cross-validation:", np.mean(specificities))

Average Sensitivity after cross-validation: 0.58
Average Specificity after cross-validation: 0.57
